For this homework, make sure that you format your notbook nicely and cite all sources in the appropriate sections. Programmatically generate or embed any figures or graphs that you need.

Names: __YOUR NAMES HERE__

Step 1: Train your own word embeddings
--------------------------------

(describe the provided dataset that you have chosen here)

Describe what data set you have chosen to compare and contrast with the your chosen provided dataset. Make sure to describe where it comes from and it's general properties.

(describe your dataset here)

In [29]:
# import your libraries here
import nltk
import pandas as pd
import keras
import numpy as np
from gensim.models import KeyedVectors

### a) Train embeddings on GIVEN dataset

In [30]:
# code to train your word embeddings

SPOOKY_AUTHOR_TEXT = "spooky-author-identification/train.csv"

# Read the file and prepare the training data 
# so that it is in the following format
spooky_df = pd.read_csv(SPOOKY_AUTHOR_TEXT)

# data = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
# 			['this', 'is', 'the', 'second', 'sentence'],
# 			['yet', 'another', 'sentence'],
# 			['one', 'more', 'sentence'],
# 			['and', 'the', 'final', 'sentence']]

spooky_df

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
...,...,...,...
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP


In [31]:
from gensim.models import Word2Vec

# make all the text lowercase
spooky_df.text = spooky_df.text.str.lower()

# save the data as a list of tokenized sentences
data = spooky_df.text.apply(nltk.word_tokenize).tolist()

# The dimension of word embedding. 
# This variable will be used throughout the program
# you may vary this as you desire
EMBEDDINGS_SIZE = 200

# Train the Word2Vec model from Gensim. 
# Below are the hyperparameters that are most relevant. 
# But feel free to explore other 
# options too:
sg = 1
window = 5
vector_size = EMBEDDINGS_SIZE
min_count = 1

model_spooky = Word2Vec(sentences=data, vector_size=vector_size, window=window, min_count=min_count, sg=sg)

In [32]:
# if you save your Word2Vec as the variable model, this will 
# print out the vocabulary size
print('Vocab size: {}'.format(len(model_spooky.wv)))

Vocab size: 25372


In [33]:
# You can save file in txt format, then load later if you wish.
model_spooky.wv.save_word2vec_format('embeddings.txt', binary=False)

### b) Train embedding on YOUR dataset

In [34]:
from nltk.corpus import reuters
file_ids = reuters.file_ids = reuters.fileids()

all_sents = []
for file_id in file_ids:
    sent_tokens = reuters.sents(file_id)
    sentences = [" ".join(list_of_words).lower() for list_of_words in sent_tokens]
    all_sents.extend(sentences)

reuters_json = {"text": all_sents}
df_reuters = pd.DataFrame.from_dict(reuters_json)

df_reuters

,text
0,asian exporters fear damage from u . s .- japa...
1,they told reuter correspondents in asian capit...
2,but some exporters said that while the conflic...
3,the u . s . has said it will impose 300 mln dl...
4,unofficial japanese estimates put the impact o...
...,...
54711,knight - ridder inc & lt ; krn > sets quarterl...
54712,technitrol inc & lt ; tnl > sets quarterly qtl...
54713,nationwide cellular service inc & lt ; ncel > ...
54714,& lt ; a . h . a .


In [35]:
reuters_data = df_reuters.text.apply(nltk.word_tokenize).tolist()
model_reuters = Word2Vec(sentences=reuters_data, vector_size=vector_size, window=window, min_count=min_count, sg=sg)
print('Vocab size: {}'.format(len(model_reuters.wv)))

Vocab size: 30996


In [36]:
print(reuters.raw(file_ids[0]))
print(reuters.sents(file_ids[0]))

ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT
  Mounting trade friction between the
  U.S. And Japan has raised fears among many of Asia's exporting
  nations that the row could inflict far-reaching economic
  damage, businessmen and officials said.
      They told Reuter correspondents in Asian capitals a U.S.
  Move against Japan might boost protectionist sentiment in the
  U.S. And lead to curbs on American imports of their products.
      But some exporters said that while the conflict would hurt
  them in the long-run, in the short-term Tokyo's loss might be
  their gain.
      The U.S. Has said it will impose 300 mln dlrs of tariffs on
  imports of Japanese electronics goods on April 17, in
  retaliation for Japan's alleged failure to stick to a pact not
  to sell semiconductors on world markets at below cost.
      Unofficial Japanese estimates put the impact of the tariffs
  at 10 billion dlrs and spokesmen for major electronics firms
  said they would virtually halt exports

https://testanother-codwar.medium.com/sentiment-analysis-on-nltk-reuters-corpus-2feed2a695e7

What text-normalization and pre-processing did you do and why? __YOUR ANSWER HERE__

Step 2: Evaluate the differences between the word embeddings
----------------------------

(make sure to include graphs, figures, and paragraphs with full sentences)

## Write down your analysis:

Cite your sources:
-------------

Step 3: Feedforward Neural Language Model
--------------------------

### a) First, encode  your text into integers

In [37]:
# Importing utility functions from Keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

# The size of the ngram language model you want to train
# change as needed for your experiments
NGRAM = 3 


### b) Next, prepare your sequences from text

#### Fixed ngram based sequences 

In [38]:
def generate_ngram_training_samples(encoded: list[list[int]], ngram:int) -> list[list[int]]:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    up to you, we've put in what we used
    but you can add/remove as needed
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''
    # flattened list comprehension
    samples = np.array([np.array(sentence[i:i+ngram]) for sentence in encoded for i in range(len(sentence)-ngram+1)])
    return samples


### c) Then, split the sequences into X and y and create a Data Generator

In [39]:
# JUST FOR TESTING PURPOSES SINCE I DONT HAVE WORD EMBEDDINGS TO WORK WITH
"""
import gensim.downloader
EMBEDDINGS_SIZE = 100
glove_vectors = gensim.downloader.load('glove-twitter-100')
"""

"\nimport gensim.downloader\nEMBEDDINGS_SIZE = 100\nglove_vectors = gensim.downloader.load('glove-twitter-100')\n"

In [ ]:
from time import time_ns

# Note here that the sequences were in the form: 
# sequence = [x1, x2, ... , x(n-1), y]
# We still need to separate it into [[x1, x2, ... , x(n-1)], ...], [y1, y2, ...]

# returns X and y using given encoded ngram training samples
def split_X_y(samples: list[list[int]]) -> tuple[list[list[int]],list[int]]:
    return np.array([l[:-1] for l in samples]), np.array([l[-1] for l in samples])

def read_embeddings(tokenizer: Tokenizer, model: KeyedVectors) -> tuple[dict[str,list], dict[int,list]]:
    '''Loads and parses embeddings trained in earlier.
    Parameters and return values are up to you.
    '''
    
    # you may find generating the following two dicts useful:
    # word to embedding : {'the':[0....], ...}
    # index to embedding : {1:[0....], ...} 
    # use your tokenizer's word_index to find the index of
    # a given word
    word_embeddings = {w:model[w]  if w in model else np.zeros(EMBEDDINGS_SIZE) for w in tokenizer.word_index}
    encoded_embeddings = {tokenizer.word_index[w]:word_embeddings[w] for w in word_embeddings}
    return word_embeddings, encoded_embeddings

lasttime=-1

def data_generator(X, y, batch_size: int, tokenizer: Tokenizer, enc_embed:dict) -> tuple[list,list]:
    '''
    Returns data generator to be used by feed_forward
    https://wiki.python.org/moin/Generators
    https://realpython.com/introduction-to-python-generators/
    
    Yields batches of embeddings and labels to go with them.
    Use one hot vectors to encode the labels 
    (see the to_categorical function)
    
    '''
    global lasttime
    lasttime = time_ns()
    i = 0
    for i in range(0,len(y),batch_size):
        t1 = time_ns()
        next_i = min(len(y), i+batch_size)
        # for each sequence in the batch, flatten all word embedding vectors into one vector
        embeddings = np.array([np.array([weight for word_index in sequence for weight in enc_embed[word_index]]) for sequence in X[i:next_i]])
        labels = to_categorical(y[i:next_i]-1,num_classes=len(tokenizer.word_counts))
        t2 = time_ns()
        # print("elapsed time (ms):",round((t2-t1)*0.000001,6))
        # print("overall time (ms):",round((t1-lasttime)*0.000001,6))
        lasttime = time_ns()
        yield embeddings, labels




### d) Train your models

In [50]:
# given lines of text (arraylike) and embeddings (model.wv)
def train_model(corpus,embeddings:KeyedVectors):

    # Initializing a Tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)
    # encoded = tokenizer.texts_to_sequences(["Hello, my love, my world.","I love you more than the world itself."])
    encoded = tokenizer.texts_to_sequences(corpus)
    # print(encoded)
    VOCAB_SIZE = len(tokenizer.word_counts)
    print(VOCAB_SIZE)

    ngram_samples = generate_ngram_training_samples(encoded, NGRAM)
    print("ngram_samples:",ngram_samples[:4])

    X, y = split_X_y(ngram_samples)
    print("X:",X)
    print("y:",y)

    word_embed, enc_embed = read_embeddings(tokenizer,embeddings)
    # code to train a feedforward neural language model 
    # on a set of given word embeddings
    # make sure not to just copy + paste to train your two models

    num_sequences_per_batch = 64 # this is the batch size
    steps_per_epoch = len(ngram_samples)//num_sequences_per_batch  # Number of batches per epoch
    train_generator = data_generator(X, y, num_sequences_per_batch, tokenizer, enc_embed)
    sample=next(train_generator) # this is how you get data out of generators
    print(sample[0].shape, "; (n-1)*EMBEDDING_SIZE) =",(NGRAM-1)*EMBEDDINGS_SIZE) # (batch_size, (n-1)*EMBEDDING_SIZE)  (128, 200)
    print(sample[1].shape)   # (batch_size, |V|) to_categorical

    # Define the model architecture using Keras Sequential API
    model = Sequential()
    model.add(layer_i := Dense(1000, input_dim=(NGRAM-1)*EMBEDDINGS_SIZE)) 
    model.add(layer_h1 := Dense(500, activation='relu'))
    # model.add(layer_h2 := Dense(100, activation='relu'))
    model.add(layer_o := Dense(VOCAB_SIZE)) 

    # for a multi-class classification problem
    model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    

    # Start training the model
    model.fit(x=train_generator, 
            steps_per_epoch=steps_per_epoch,
            epochs=1) 



In [51]:
train_model(spooky_df["text"],model_spooky.wv)  

25943
ngram_samples: [[  26 2945  143]
 [2945  143 1372]
 [ 143 1372   22]
 [1372   22   36]]
X: [[  26 2945]
 [2945  143]
 [ 143 1372]
 ...
 [  20  465]
 [ 465    9]
 [   9    2]]
y: [ 143 1372   22 ...    9    2 5991]
overall time (ms): 0.003247
(64, 400) ; (n-1)*EMBEDDING_SIZE) = 400
(64, 25943)
overall time (ms): 49.069719
overall time (ms): 371.395961
 145/7562 [..............................] - ETA: 15:45 - loss: 11.6688 - accuracy: 0.0571

KeyboardInterrupt: 

In [ ]:
train_model(df_reuters["text"],model_reuters.wv)

### e) Generate Sentences

In [ ]:
# generate a sequence from the model
def generate_seq(model: Sequential, 
                 tokenizer: Tokenizer, 
                 seed: list, 
                 n_words: int):
    '''
    Parameters:
        model: your neural network
        tokenizer: the keras preprocessing tokenizer
        seed: [w1, w2, w(n-1)]
        n_words: generate a sentence of length n_words
    Returns: string sentence
    '''
    pass

### f) Compare your generated sentences

You may find it useful to run your HW 2 code on one of the datasets (or a subset of the dataset) that you used for this homework.

Sources Cited
----------------------------


- https://pyimagesearch.com/2021/05/06/implementing-feedforward-neural-networks-with-keras-and-tensorflow/
- https://keras.io/api/layers/core_layers/dense/